## Import Dependencies

In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.GPBO_Class_fxns import * #Fix this later

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.2 s, sys: 268 ms, total: 2.47 s
Wall time: 2.5 s


## Define Case Study & Most of Method

In [2]:
#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

cs_name  = "CS2"

#Want to make this all one array. I think that makes more sense
true_model_coefficients = np.array([-200,-100,-170,15,-1,-1,-6.5,0.7,0,0,11,0.6,-10,-10,-6.5,0.7,1,0,-0.5,-1,0,0.5,1.5,1])
param_dict = {0 : 'A_1', 1 : 'A_2', 2 : 'A_3', 3 : 'A_4', 4 : 'a_1', 5 : 'a_2', 6 : 'a_3', 7 : 'a_4', 
             8 : 'b_1', 9 : 'b_2', 10 : 'b_3', 11 : 'b_4', 12 : 'c_1', 13 : 'c_2', 14 : 'c_3', 15 : 'c_4',
             16 : 'x0_1', 17 : 'x0_2', 18 : 'x0_3', 19 : 'x0_4', 20 : 'x1_1', 21 : 'x1_2', 22 : 'x1_3', 23 : 'x1_4'}

indecies_to_consider = list(param_dict.keys())[4:12] #This is what changes for different subproblems of CS2
true_params = true_model_coefficients[indecies_to_consider]
# print([param_dict[key] if key in indecies_to_consider else f"No value found for key {key}" for key in indecies_to_consider])
ep0 = 1
sep_fact = 1.0
normalize = False
num_x_data = 27
num_theta_data = 20
lhs_gen_theta = True
eval_all_pairs = False
package = "scikit_learn"
noise_mean = 0
noise_std = 0.01
kernel = "Mat_52"
set_lenscl = 1
outputscl = False
retrain_GP = 2
GP_train_iter = 300
bo_iter_tot = 3
bo_run_tot = 2
save_fig = False
save_data = False
x_data_vals = None
num_data = None
seed = 1
CS = CaseStudyParameters(cs_name, true_params, true_model_coefficients, param_dict, 
                         indecies_to_consider, ep0, sep_fact, normalize, num_x_data, num_theta_data, 
                         num_data, lhs_gen_theta, x_data_vals, eval_all_pairs, package, noise_mean, noise_std, kernel, 
                         set_lenscl, outputscl, retrain_GP, GP_train_iter, bo_iter_tot, bo_run_tot, save_fig, save_data, 
                         DateTime, seed)

#Generate Method (except last part)
meth_name = "1A"
emulator = False
obj = "obj"
sparse_grid = False

method = GPBO_Methods(meth_name, emulator, obj, sparse_grid, None)

## Find Experimental X Data (If not given)

In [3]:
driver = GPBO_Driver(CS)
bounds_x = np.array([[-1.5, -0.5], [1, 2]])
gen_meth = "LHS"
seed = 1
x_data = clean_1D_arrays(driver.create_param_data(num_x_data, bounds_x, gen_meth))
exp_data = Data(None, true_params, x_data, None, None, None, None, None, None)
CS.x_data_vals = exp_data.x_vals
# print(x_data)

## Normalize Experimental X Data (optional)

In [4]:
bounds_p = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                    [ 2,  2,   0,  2,  2,  2, 15,  2]])
scaled_exp_data = Data(None, true_params, x_data, None, None, None, None, None, None)
scaled_exp_data.theta_true = scaled_exp_data.normalize(true_params, bounds_p)
scaled_exp_data.x_data = scaled_exp_data.normalize(x_data, bounds_x)

# print(scaled_exp_data.theta_true)
# print(scaled_exp_data.x_data)

## Generate Experimental y Data

In [5]:
#Best way to differentiate between case studies? Name? How to name appropriately?
#Note this is a new way of defining the function for CS2 which I think will be better suited to our needs.
y_exp = driver.create_y_exp_data()
exp_data.y_vals = y_exp
# print(exp_data.y_vals)

## Generate Simulation Theta Data

In [6]:
#Rather than using a skip param_type, I will be using indecies and dictionaries.
sim_data = Data(None, true_params, x_data, None, None, None, None, None, None)
bounds_p = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                    [ 2,  2,   0,  2,  2,  2, 15,  2]])

##Just generate some data for now and fix the simulation data creators.
# Define the lower and upper bounds
sim_data = Data(None, true_params, x_data, None, None, None, None, None, None)
sim_data.theta_vals = clean_1D_arrays(driver.create_param_data(num_theta_data, bounds_p, gen_meth))
# print(sim_data.x_vals)

## Generate y_sim, sse, or log(sse) data (Run Simulations)

In [7]:
#Set number of training dimensions for the method
method.GP_training_dims = method.set_GP_training_dims(sim_data.x_vals, sim_data.theta_vals)
# print(method.GP_training_dims)

#Create y_sim data based on method, data, and case study
y_sim = driver.create_sim_data(method, sim_data, exp_data)
sim_data.y_vals = y_sim
# print(y_sim)

AttributeError: 'str' object has no attribute 'obj'

## Augmenting Training Data (Skipped Steps)

In [ ]:
best_theta = Data(true_params.reshape(1,-1), true_params, x_data, None, None, None, None, None, None)
best_theta.y_vals = driver.create_sim_data(method, best_theta, exp_data)

print(len(sim_data.theta_vals), len(sim_data.y_vals))

sim_data = driver.augment_train_data(sim_data, best_theta)

print(len(sim_data.theta_vals), len(sim_data.y_vals))